# Beam Pairing Notebook

In [1]:
import h5py
import simplekml
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from osgeo import ogr
import fiona

# Packages for kml2df (temporary addition for testing)
import os
import numpy as np
import pandas as pd
import re
import datetime

/tmp/ipykernel_389/3567945044.py:4: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [3]:
# Read collection of RGT tracks
rgt_paths = '/home/jovyan/shared-public/ICESat-2-Hackweek/ground_tracks/cycle21_1hz_points.csv'
rgt_tracks = pd.read_csv(rgt_paths)
rgt_tracks

,Unnamed: 0,lat,lon,rgt,timestamp,time_str,geometry
0,0,0.027959,-0.131847,1,1.695089e+09,Tue 2023-Sep-19 02:03:28,POINT (-0.1318471781239014 0.0279589282517555)
1,1,0.091968,-0.138250,1,1.695089e+09,Tue 2023-Sep-19 02:03:29,POINT (-0.1382496846602931 0.0919676662034125)
2,2,0.155977,-0.144652,1,1.695089e+09,Tue 2023-Sep-19 02:03:30,POINT (-0.1446522061867684 0.1559766013588329)
3,3,0.219986,-0.151055,1,1.695089e+09,Tue 2023-Sep-19 02:03:31,POINT (-0.1510547481925733 0.2199857315599926)
4,4,0.283995,-0.157457,1,1.695089e+09,Tue 2023-Sep-19 02:03:32,POINT (-0.1574573161672413 0.2839950546486701)
...,...,...,...,...,...,...,...
7845397,5651,-0.347574,-0.094110,1387,1.702936e+09,Mon 2023-Dec-18 21:43:11,POINT (-0.0941102460933371 -0.3475744048835365)
7845398,5652,-0.283567,-0.100513,1387,1.702936e+09,Mon 2023-Dec-18 21:43:12,POINT (-0.1005128153132467 -0.2835669445179809)
7845399,5653,-0.219559,-0.106915,1387,1.702936e+09,Mon 2023-Dec-18 21:43:13,POINT (-0.106915367320209 -0.219559274377078)
7845400,5654,-0.155551,-0.113318,1387,1.702936e+09,Mon 2023-Dec-18 21:43:14,POINT (-0.1133179076161863 -0.155551396617297)


In [28]:
# Approximate location of left and right beams
rgt_tracks['right_lon'] = rgt_tracks['lon'] + 3300./(np.cos(np.radians(rgt_tracks['lat']))*111000)
rgt_tracks['left_lon'] = rgt_tracks['lon'] - 3300./(np.cos(np.radians(rgt_tracks['lat']))*111000)
rgt_tracks.head()

,Unnamed: 0,lat,lon,rgt,timestamp,time_str,geometry,right_lon,left_lon
0,0,0.027959,-0.131847,1,1.695089e+09,Tue 2023-Sep-19 02:03:28,POINT (-0.1318471781239014 0.0279589282517555),-0.102117,-0.161577
1,1,0.091968,-0.138250,1,1.695089e+09,Tue 2023-Sep-19 02:03:29,POINT (-0.1382496846602931 0.0919676662034125),-0.108520,-0.167979
2,2,0.155977,-0.144652,1,1.695089e+09,Tue 2023-Sep-19 02:03:30,POINT (-0.1446522061867684 0.1559766013588329),-0.114922,-0.174382
3,3,0.219986,-0.151055,1,1.695089e+09,Tue 2023-Sep-19 02:03:31,POINT (-0.1510547481925733 0.2199857315599926),-0.121325,-0.180785
4,4,0.283995,-0.157457,1,1.695089e+09,Tue 2023-Sep-19 02:03:32,POINT (-0.1574573161672413 0.2839950546486701),-0.127727,-0.187187


In [59]:
# Generate Point geometries for each of the beam pairs
from shapely import Point, MultiPoint
rgt_tracks['geom1'] = [Point(x, y) for x, y in zip(rgt_tracks.lon, rgt_tracks.lat)]
rgt_tracks['geom2'] = [Point(x, y) for x, y in zip(rgt_tracks.right_lon, rgt_tracks.lat)]
rgt_tracks['geom3'] = [Point(x, y) for x, y in zip(rgt_tracks.left_lon, rgt_tracks.lat)]
rgt_tracks.head()

,Unnamed: 0,lat,lon,rgt,timestamp,time_str,geometry,right_lon,left_lon,geom2,geom3,geom1
0,0,0.027959,-0.131847,1,1.695089e+09,Tue 2023-Sep-19 02:03:28,POINT (-0.1318471781239014 0.0279589282517555),-0.102117,-0.161577,POINT (-0.1021174448545505 0.0279589282517554),POINT (-0.1615769113932523 0.0279589282517554),POINT (-0.1318471781239014 0.0279589282517554)
1,1,0.091968,-0.138250,1,1.695089e+09,Tue 2023-Sep-19 02:03:29,POINT (-0.1382496846602931 0.0919676662034125),-0.108520,-0.167979,POINT (-0.1085199166316459 0.0919676662034124),POINT (-0.1679794526889403 0.0919676662034124),POINT (-0.1382496846602931 0.0919676662034124)
2,2,0.155977,-0.144652,1,1.695089e+09,Tue 2023-Sep-19 02:03:30,POINT (-0.1446522061867684 0.1559766013588329),-0.114922,-0.174382,POINT (-0.1149223662939896 0.1559766013588329),POINT (-0.1743820460795472 0.1559766013588329),POINT (-0.1446522061867684 0.1559766013588329)
3,3,0.219986,-0.151055,1,1.695089e+09,Tue 2023-Sep-19 02:03:31,POINT (-0.1510547481925733 0.2199857315599926),-0.121325,-0.180785,POINT (-0.1213247993300404 0.2199857315599926),POINT (-0.1807846970551062 0.2199857315599926),POINT (-0.1510547481925733 0.2199857315599926)
4,4,0.283995,-0.157457,1,1.695089e+09,Tue 2023-Sep-19 02:03:32,POINT (-0.1574573161672413 0.2839950546486701),-0.127727,-0.187187,POINT (-0.1277272212283182 0.2839950546486701),POINT (-0.1871874111061644 0.2839950546486701),POINT (-0.1574573161672413 0.2839950546486701)


In [60]:
# Look at only a single RGT
single_rgt_track = rgt_tracks[rgt_tracks['rgt']==1356]
single_rgt_track.head()

,Unnamed: 0,lat,lon,rgt,timestamp,time_str,geometry,right_lon,left_lon,geom2,geom3,geom1
7664398,0,0.051389,35.683078,1356,1.702755e+09,Sat 2023-Dec-16 19:26:01,POINT (35.6830782312669 0.0513888778172799),35.712808,35.653348,POINT (35.71280797295451 0.0513888778172799),POINT (35.65334848957929 0.0513888778172799),POINT (35.6830782312669 0.0513888778172799)
7664399,1,0.115399,35.676678,1356,1.702755e+09,Sat 2023-Dec-16 19:26:02,POINT (35.67667812014548 0.115398857221975),35.706408,35.646948,POINT (35.70640791017551 0.115398857221975),POINT (35.64694833011546 0.115398857221975),POINT (35.67667812014548 0.115398857221975)
7664400,2,0.179409,35.670278,1356,1.702755e+09,Sat 2023-Dec-16 19:26:03,POINT (35.67027799160195 0.1794090299600341),35.700008,35.640548,POINT (35.70000786708084 0.1794090299600341),POINT (35.64054811612306 0.1794090299600341),POINT (35.67027799160195 0.1794090299600341)
7664401,3,0.243419,35.663878,1356,1.702755e+09,Sat 2023-Dec-16 19:26:04,POINT (35.66387784013344 0.2434193938711747),35.693608,35.634148,POINT (35.69360783816851 0.2434193938711747),POINT (35.634147842098365 0.2434193938711747),POINT (35.66387784013344 0.2434193938711747)
7664402,4,0.307430,35.657478,1356,1.702755e+09,Sat 2023-Dec-16 19:26:05,POINT (35.6574776602617 0.3074299467948653),35.687208,35.627748,POINT (35.68720781796135 0.3074299467948653),POINT (35.627747502562045 0.3074299467948653),POINT (35.6574776602617 0.3074299467948653)


In [70]:
# Combine Point geometries into Multipoint object
single_rgt_track['multi2'] = [MultiPoint([x, y, z]) for x, y, z in zip(single_rgt_track.geom1, 
                                                                 single_rgt_track.geom2,
                                                                 single_rgt_track.geom3)]
single_rgt_track.head()

/tmp/ipykernel_389/2075605559.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_rgt_track['multi'] = [MultiPoint([x, y]) for x, y in zip(single_rgt_track.geom2, single_rgt_track.geom3)]
/tmp/ipykernel_389/2075605559.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_rgt_track['multi2'] = [MultiPoint([x, y, z]) for x, y, z in zip(single_rgt_track.geom1,


,Unnamed: 0,lat,lon,rgt,timestamp,time_str,geometry,right_lon,left_lon,geom2,geom3,geom1,multi,multi2
7664398,0,0.051389,35.683078,1356,1.702755e+09,Sat 2023-Dec-16 19:26:01,POINT (35.6830782312669 0.0513888778172799),35.712808,35.653348,POINT (35.71280797295451 0.0513888778172799),POINT (35.65334848957929 0.0513888778172799),POINT (35.6830782312669 0.0513888778172799),MULTIPOINT (35.71280797295451 0.05138887781727...,MULTIPOINT (35.6830782312669 0.051388877817279...
7664399,1,0.115399,35.676678,1356,1.702755e+09,Sat 2023-Dec-16 19:26:02,POINT (35.67667812014548 0.115398857221975),35.706408,35.646948,POINT (35.70640791017551 0.115398857221975),POINT (35.64694833011546 0.115398857221975),POINT (35.67667812014548 0.115398857221975),MULTIPOINT (35.70640791017551 0.11539885722197...,MULTIPOINT (35.67667812014548 0.11539885722197...
7664400,2,0.179409,35.670278,1356,1.702755e+09,Sat 2023-Dec-16 19:26:03,POINT (35.67027799160195 0.1794090299600341),35.700008,35.640548,POINT (35.70000786708084 0.1794090299600341),POINT (35.64054811612306 0.1794090299600341),POINT (35.67027799160195 0.1794090299600341),MULTIPOINT (35.70000786708084 0.17940902996003...,MULTIPOINT (35.67027799160195 0.17940902996003...
7664401,3,0.243419,35.663878,1356,1.702755e+09,Sat 2023-Dec-16 19:26:04,POINT (35.66387784013344 0.2434193938711747),35.693608,35.634148,POINT (35.69360783816851 0.2434193938711747),POINT (35.634147842098365 0.2434193938711747),POINT (35.66387784013344 0.2434193938711747),MULTIPOINT (35.69360783816851 0.24341939387117...,MULTIPOINT (35.66387784013344 0.24341939387117...
7664402,4,0.307430,35.657478,1356,1.702755e+09,Sat 2023-Dec-16 19:26:05,POINT (35.6574776602617 0.3074299467948653),35.687208,35.627748,POINT (35.68720781796135 0.3074299467948653),POINT (35.627747502562045 0.3074299467948653),POINT (35.6574776602617 0.3074299467948653),MULTIPOINT (35.68720781796135 0.30742994679486...,MULTIPOINT (35.6574776602617 0.307429946794865...


In [78]:
# Set the MultiPoint geometry, and remove redundant columns
single_track_gdf = single_rgt_track.set_geometry('multi2')
single_track_gdf = single_track_gdf.drop(['geometry','geom1','geom2', 'geom3', 'multi'], axis=1)

In [85]:
# Save geodataframe to a geoJSON
single_track_gdf.to_file('rgt1356_allbeams.geojson', driver='GeoJSON')

In [86]:
# Read the geoJSON back in
data = gpd.read_file('rgt1356_allbeams.geojson')

In [87]:
# View CRS of track information
data.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [89]:
# View ICESat-2 tracks over ESRI map
data.explore(tiles='Esri.WorldImagery')

In [90]:
data.head()

,Unnamed: 0,lat,lon,rgt,timestamp,time_str,right_lon,left_lon,geometry
0,0,0.051389,35.683078,1356,1.702755e+09,Sat 2023-Dec-16 19:26:01,35.712808,35.653348,"MULTIPOINT (35.68308 0.05139, 35.71281 0.05139..."
1,1,0.115399,35.676678,1356,1.702755e+09,Sat 2023-Dec-16 19:26:02,35.706408,35.646948,"MULTIPOINT (35.67668 0.11540, 35.70641 0.11540..."
2,2,0.179409,35.670278,1356,1.702755e+09,Sat 2023-Dec-16 19:26:03,35.700008,35.640548,"MULTIPOINT (35.67028 0.17941, 35.70001 0.17941..."
3,3,0.243419,35.663878,1356,1.702755e+09,Sat 2023-Dec-16 19:26:04,35.693608,35.634148,"MULTIPOINT (35.66388 0.24342, 35.69361 0.24342..."
4,4,0.307430,35.657478,1356,1.702755e+09,Sat 2023-Dec-16 19:26:05,35.687208,35.627748,"MULTIPOINT (35.65748 0.30743, 35.68721 0.30743..."
